# animation
after running the verification experiment tutorial_plume_on_slope in "myrun" with 
 useSingleCPUio = .TRUE.,
 dumpInitAndLast = .FALSE.,
 monitorFreq = 7200.,
 dumpFreq = 7200.,
and for 2 days (8640 timesteps), run this notebook to plot temperature "theta"

In [5]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from xmitgcm import open_mdsdataset
# helper function
def sq(a):
    a = np.squeeze(a)
    return np.ma.masked_array(a==0.,a)

bdir = '/Users/mlosch/Documents/teaching/GLOMAR/AdvancedOceanModelling/MITgcm/verification'
experiment0 = 'tutorial_plume_on_slope/run00'
experiment1 = 'tutorial_plume_on_slope/run01'

In [8]:
import warnings
warnings.filterwarnings('ignore')
# suppress numerous warnings only when reading data into the workspace with xmitgcm.open_mdsdataset
with warnings.catch_warnings():
    # this will suppress all warnings in this block
    warnings.simplefilter("ignore")
    ds0 = open_mdsdataset(os.path.join(bdir,experiment0),delta_t=20.,ref_date="2001-1-1 0:0:0",
                          prefix='T',geometry='cartesian')
    ds1 = open_mdsdataset(os.path.join(bdir,experiment1),delta_t=20.,ref_date="2001-1-1 0:0:0",
                          prefix='T',geometry='cartesian')

#os.path.join(bdir,experiment0), ds0

In [7]:
mytime = -1
mycmap = plt.cm.viridis

plt.close('all')
fig1,ax1 = plt.subplots(2,1,sharex=True,sharey=True,figsize=(10,5))
# replace zeros by NaN
theta0=ds0.T.where(ds0.T!=0)
theta1=ds1.T.where(ds1.T!=0)
# set up the plot for t=0
k = 0
botcolor='lightgrey'
ph0 = theta0.isel(time=k).plot(ax=ax1[0],cmap=mycmap,vmin=-0.08,vmax=0.02)
ph1 = theta1.isel(time=k).plot(ax=ax1[1],cmap=mycmap,vmin=-0.08,vmax=0.02)
# overlay topography as a step function to make plot look nicer
ax1[0].fill_between(ds0.XC,-ds0.Depth[0,:],-ds0.Depth.max(),step='mid',facecolor=botcolor)
ax1[1].fill_between(ds1.XC,-ds1.Depth[0,:],-ds1.Depth.max(),step='mid',facecolor=botcolor)
ax1[0].title.set_text('iter = %i, time = %s'%(theta0.iter[k],np.datetime_as_string(theta0.time[k], unit='s')))
plt.tight_layout()
# define a function that updates data in plot for each time slice
def animate(k):
    ph0.set_array(theta0.isel(time=k).data.ravel())
    ph1.set_array(theta1.isel(time=k).data.ravel())
    ax1[0].title.set_text('iter = %i, time = %s'%(theta0.iter[k],np.datetime_as_string(theta0.time[k], unit='s')))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
#run the animation
anim = FuncAnimation(
    fig1, animate, interval=100, frames=len(theta1.time)-1)
 
anim.save('plume_on_slope.mp4')
fig1.show()